# STATIC GRAPH

In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

## TODO

- [x] Dynamic batch size
- [x] Mask padding while calculating loss
- [x] Deal with variable length input sequence
- [ ] Add `EOS` (end of sentence) token to data (target sequence)

## DATA

In [2]:
import data_utils
metadata, idx_q, idx_a = data_utils.load_data('../data/')

In [3]:
# add special symbol
i2w = metadata['idx2w']
w2i = metadata['w2idx']

## Parameters

In [31]:
B = 256
L = len(idx_q[0])
vocab_size = len(i2w)
enc_hdim = 250
dec_hdim = enc_hdim

## Graph

In [14]:
tf.reset_default_graph()

### Placeholders

In [15]:
inputs = tf.placeholder(tf.int32, shape=[None,L], name='inputs')
targets = tf.placeholder(tf.int32, shape=[None,L], name='targets')
go_token = tf.reshape(tf.fill(tf.shape(inputs[:,0]), w2i['GO']), [-1,1])
decoder_inputs = tf.concat(
    values=[go_token, targets[:, 1:]],
    axis=1)
training = tf.placeholder(tf.bool, name='is_training')
batch_size = tf.shape(inputs)[0] # infer batch size

In [16]:
decoder_inputs, targets, inputs, batch_size

(<tf.Tensor 'concat:0' shape=(?, 21) dtype=int32>,
 <tf.Tensor 'targets:0' shape=(?, 21) dtype=int32>,
 <tf.Tensor 'inputs:0' shape=(?, 21) dtype=int32>,
 <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>)

## Sequence Length

In [17]:
# Encoder
#  Get sequence lengths
def seq_len(t):
    return tf.reduce_sum(tf.cast(t>0, tf.int32), axis=1)

enc_seq_len = seq_len(inputs)

In [18]:
# Decoder
#   Mask targets
#    to be applied to cross entropy loss
padding_mask = tf.cast(targets > 0, tf.float32 )

## Embedding

In [19]:
emb_mat = tf.get_variable('emb', shape=[vocab_size, enc_hdim], dtype=tf.float32, 
                         initializer=xinit())
emb_enc_inputs = tf.nn.embedding_lookup(emb_mat, inputs)
emb_dec_inputs = tf.nn.embedding_lookup(emb_mat, decoder_inputs)

## Encoder

In [20]:
with tf.variable_scope('encoder'):
    encoder_cell = tf.contrib.rnn.LSTMCell(num_units=enc_hdim)
    encoder_outputs, context = tf.nn.dynamic_rnn(cell=encoder_cell, # encoder cell
                          inputs=emb_enc_inputs, # embedding input sequences
                          sequence_length=enc_seq_len, # lengths of sequences
                          initial_state = # zero state of cell
                          encoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
                         )

## Decoder

In [21]:
# inference - custom rnn
def inference_decoder(cell, state):
    #tf.get_variable_scope().reuse_variables()
    with tf.variable_scope('decoder_1'):
        dec_outputs = []
        #input_ = tf.constant(w2i['GO'], dtype=tf.int32, shape=[B,])
        input_ = tf.nn.embedding_lookup(emb_mat, tf.squeeze(go_token))
        input_.set_shape([None, dec_hdim])
        
        for i in range(L):
            decoder_output, state = cell(input_, state)
            dec_outputs.append(decoder_output)
            input_ = decoder_output

    return tf.stack(dec_outputs), None

In [22]:
with tf.variable_scope('decoder') as scope:
    decoder_cell = tf.contrib.rnn.LSTMCell(num_units=dec_hdim)
    
    decoder_outputs, _ = tf.nn.dynamic_rnn(cell=decoder_cell,
                          inputs=emb_dec_inputs, 
                          initial_state= context,
                          sequence_length= seq_len(decoder_inputs),
                          scope='decoder_1')
                         
    tf.get_variable_scope().reuse_variables()
    
    decoder_outputs_inf, _ = inference_decoder(decoder_cell, context)

## Logits and Probabilities

In [23]:
Wo = tf.get_variable('Wo', shape=[dec_hdim, vocab_size], dtype=tf.float32, 
                         initializer=xinit())
bo = tf.get_variable('bo', shape=[vocab_size], dtype=tf.float32, 
                         initializer=xinit())
proj_outputs = tf.matmul(tf.reshape(decoder_outputs, [-1, dec_hdim]), Wo) + bo
proj_outputs_inf = tf.matmul(tf.reshape(decoder_outputs_inf, [-1, dec_hdim]), Wo) + bo

logits = tf.cond(tf.random_normal(shape=()) > 0.,
    lambda : tf.reshape(proj_outputs, [batch_size, L, vocab_size]),
    lambda : tf.reshape(proj_outputs_inf, [batch_size, L, vocab_size])
                )
probs = tf.nn.softmax(tf.reshape(proj_outputs_inf, [batch_size, L, vocab_size]))

## Loss

In [24]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits =  logits,
    labels = targets)
# apply mask
masked_cross_entropy = cross_entropy * padding_mask
# average across sequence, batch
loss = tf.reduce_mean(cross_entropy)

## Optimizatin

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

## Inference

In [26]:
prediction = tf.argmax(probs, axis=-1)

# TRAINING

In [27]:
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.InteractiveSession(config = config)
sess.run(tf.global_variables_initializer())

## Training parameters

In [28]:
num_epochs = 20

## Start Training

In [29]:
for i in range(num_epochs):
    avg_loss = 0.
    for j in range(len(idx_q)//B):
        _, loss_v = sess.run([train_op, loss], feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })
        avg_loss += loss_v
        if j and j%30==0:
            print('{}.{} : {}'.format(i,j,avg_loss/30))
            avg_loss = 0.

0.30 : 6.950666356086731
0.60 : 5.824809837341308
0.90 : 5.240952897071838
0.120 : 5.031051023801168
0.150 : 4.964294258753458
0.180 : 4.844659233093262
0.210 : 4.471138985951741
0.240 : 4.393439292907715
0.270 : 4.136003883679708
0.300 : 4.027467401822408
0.330 : 3.924912619590759
0.360 : 3.991020615895589
0.390 : 3.8489135026931764
0.420 : 3.781740593910217
0.450 : 3.5467817783355713


KeyboardInterrupt: 

### Compare CE and masked CE

In [ ]:
ce, mce = sess.run([cross_entropy, masked_cross_entropy], feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })

In [ ]:
ce[0], mce[0]

## Test inference

In [ ]:
j = 117
pred_v = sess.run(prediction, feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            #targets : idx_a[j*B:(j+1)*B]
        })

In [ ]:
pred_v[0], idx_a[j*B:(j+1)*B][0]

In [ ]:
def arr2sent(arr):
    return ' '.join([i2w[item] for item in arr])

In [ ]:
for i in range(B):
    print(arr2sent(pred_v[i]))

In [ ]:
arr2sent( idx_a[j*B:(j+1)*B][11])